In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
subject = 'Tocantins - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [18]:
data = pd.read_csv('2003_mo_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH,Tocantins - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.243830,1.593908e+07,1.457380e+06,10.276585,2.572692e+07,12.733000,0.700107,17.555
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.237931,1.596108e+07,1.459029e+06,10.281590,2.576177e+07,12.733000,0.700487,14.976
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.232033,1.598308e+07,1.460678e+06,10.286595,2.579662e+07,12.733000,0.700866,13.334
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.226135,1.600508e+07,1.462328e+06,10.291600,2.583147e+07,12.733000,0.701246,15.173
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.220236,1.602709e+07,1.463977e+06,10.296605,2.586632e+07,12.733000,0.701626,18.728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.097178,NaN,54.154
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.077411,NaN,55.664
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.563807,NaN,52.320
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.682099,NaN,43.036


In [19]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.738973,-1.644217,0.633315,-2.414213,0.326404,-1.246730,-2.226834
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.745082,-1.628369,0.646316,-2.377931,0.345573,-1.246730,-2.188140
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.751192,-1.612520,0.659318,-2.341648,0.364742,-1.246730,-2.149447
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.757301,-1.596671,0.672320,-2.305365,0.383911,-1.246730,-2.110754
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.763410,-1.580822,0.685321,-2.269082,0.403080,-1.246730,-2.072060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.745989,1.219532,-0.763386,1.120755,-0.578577,1.289207,1.307164
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.733597,1.213944,-0.756340,1.118160,-0.567586,1.310568,1.296751
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.721205,1.208357,-0.749293,1.115565,-0.556595,1.345357,1.286338
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.708813,1.202769,-0.742246,1.112970,-0.545604,1.340245,1.275925


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Tocantins - Consumo de Cimento (t), Length: 240, dtype: float64

In [21]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.738973,-1.644217,0.633315,-2.414213,0.326404,-1.246730,-2.226834
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.745082,-1.628369,0.646316,-2.377931,0.345573,-1.246730,-2.188140
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.751192,-1.612520,0.659318,-2.341648,0.364742,-1.246730,-2.149447
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.757301,-1.596671,0.672320,-2.305365,0.383911,-1.246730,-2.110754
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.763410,-1.580822,0.685321,-2.269082,0.403080,-1.246730,-2.072060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.443839,1.208594,-1.016210,1.001982,-0.899654,0.819052,1.403526
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.400830,1.213994,-1.006113,1.012925,-0.888702,0.823544,1.404858
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.357822,1.219395,-0.996016,1.023868,-0.877750,0.828158,1.406191
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.314814,1.224795,-0.985919,1.034811,-0.866797,0.835894,1.407523


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
157    33.787
158    41.594
159    36.019
160    45.143
161    41.704
Name: Tocantins - Consumo de Cimento (t), Length: 162, dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.228797,1.235595,-0.965724,1.056697,-0.844893,0.895407,1.410188
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.185788,1.240995,-0.955627,1.067640,-0.833940,0.923985,1.411521
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.142780,1.246395,-0.945530,1.078583,-0.822988,0.960211,1.412853
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.099772,1.251796,-0.935433,1.089526,-0.812036,1.007466,1.414186
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.056763,1.257196,-0.925336,1.100469,-0.801083,1.044647,1.415518
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.013755,1.262596,-0.915238,1.111412,-0.790131,1.067158,1.416851
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,0.970746,1.267996,-0.905141,1.122355,-0.779179,1.091594,1.418183
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,0.959245,1.267217,-0.897439,1.123736,-0.768873,1.098053,1.415006
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.947744,1.266438,-0.889737,1.125116,-0.758568,1.142705,1.411828
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,0.936243,1.265658,-0.882034,1.126496,-0.748262,1.145305,1.408651


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    45.710
163    45.564
164    43.754
165    48.554
166    40.896
167    35.255
168    34.018
169    22.422
170    29.555
171    28.919
172    25.589
173    40.237
174    37.012
175    38.793
176    38.281
177    39.711
178    33.695
179    28.206
180    31.615
181    30.275
182    29.568
183    26.940
184    35.910
185    33.954
186    37.983
187    40.046
188    32.647
189    39.065
190    36.180
191    28.749
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [25]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [26]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,target_val),
#                         validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2160468047, 1549785310, 2057921887, 2608290575, 430777764, 770102059, 2246003835, 3228140615, 1981239973, 4267925030, 4079398976, 2401865550, 3158772662, 3213177608, 3781142417, 2072016967, 1964421941, 838810575, 2133014311, 1049473340, 81037359, 408037990, 3207073327, 1952340522, 3031931658]


Step: 0 ___________________________________________
val_loss: 14.491850852966309
winner_seed: 2160468047


Step: 1 ___________________________________________
val_loss: 15.089778900146484


Step: 2 ___________________________________________
val_loss: 14.69387149810791


Step: 3 ___________________________________________
val_loss: 15.228826522827148


Step: 4 ___________________________________________
val_loss: 18.3131046295166


Step: 5 ___________________________________________
val_loss: 12.252340316772461
winner_seed: 770102059


Step: 6 ___________________________________________
val_loss: 14.690032958984375


Step: 7 ___________________________________________
val_loss: 19.7788639068603

In [29]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 21ms/step - loss: 1264.3976 - val_loss: 1515.0764
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 1392.0093 - val_loss: 1869.2365
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 1464.7572 - val_loss: 804.3021
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 285.4718 - val_loss: 75.4888
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 65.3682 - val_loss: 59.7612
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 48.7008 - val_loss: 43.9326
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 47.9823 - val_loss: 45.1796
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 42.5608 - val_loss: 84.5706
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 75.6077 - val_loss: 39.8403
Epoch 10/10000
5/5 [==============================] - 0s 4ms/step - loss: 45.5

5/5 [==============================] - 0s 4ms/step - loss: 40.2272 - val_loss: 43.9302
Epoch 82/10000
5/5 [==============================] - 0s 4ms/step - loss: 43.1217 - val_loss: 64.5606
Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 52.8665 - val_loss: 46.6570
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 39.6480 - val_loss: 44.7351
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 41.0199 - val_loss: 44.4697
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 38.5975 - val_loss: 50.5263
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 50.5920 - val_loss: 44.6464
Epoch 88/10000
5/5 [==============================] - 0s 4ms/step - loss: 38.1136 - val_loss: 51.6104
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 38.4569 - val_loss: 43.3761
Epoch 90/10000
5/5 [==============================] - 0s 4ms/step - loss: 40.3896 - val_loss: 60.

Epoch 161/10000
5/5 [==============================] - 0s 4ms/step - loss: 40.7128 - val_loss: 37.7657
Epoch 162/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.6800 - val_loss: 46.6997
Epoch 163/10000
5/5 [==============================] - 0s 4ms/step - loss: 39.2461 - val_loss: 45.7447
Epoch 164/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.8805 - val_loss: 37.9568
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 35.9262 - val_loss: 51.6896
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.6977 - val_loss: 85.9731
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 43.1133 - val_loss: 60.8298
Epoch 168/10000
5/5 [==============================] - 0s 4ms/step - loss: 38.8044 - val_loss: 44.4010
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.1198 - val_loss: 38.4779
Epoch 170/10000
5/5 [==============================] - 0s 4ms/step - loss

5/5 [==============================] - 0s 4ms/step - loss: 34.2840 - val_loss: 36.2191
Epoch 241/10000
5/5 [==============================] - 0s 4ms/step - loss: 35.5812 - val_loss: 40.4492
Epoch 242/10000
5/5 [==============================] - 0s 4ms/step - loss: 31.9441 - val_loss: 40.4482
Epoch 243/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.0046 - val_loss: 44.5048
Epoch 244/10000
5/5 [==============================] - 0s 4ms/step - loss: 37.6140 - val_loss: 44.1523
Epoch 245/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.6925 - val_loss: 34.2649
Epoch 246/10000
5/5 [==============================] - 0s 4ms/step - loss: 40.8354 - val_loss: 53.8488
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 44.5795 - val_loss: 44.5796
Epoch 248/10000
5/5 [==============================] - 0s 4ms/step - loss: 36.0240 - val_loss: 34.3301
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 32.6344 - val_

5/5 [==============================] - 0s 4ms/step - loss: 33.3949 - val_loss: 32.1043
Epoch 320/10000
5/5 [==============================] - 0s 4ms/step - loss: 31.4947 - val_loss: 34.7065
Epoch 321/10000
5/5 [==============================] - 0s 4ms/step - loss: 30.8460 - val_loss: 32.3780
Epoch 322/10000
5/5 [==============================] - 0s 4ms/step - loss: 31.4603 - val_loss: 41.8505
Epoch 323/10000
5/5 [==============================] - 0s 4ms/step - loss: 32.1968 - val_loss: 32.2247
Epoch 324/10000
5/5 [==============================] - 0s 4ms/step - loss: 33.1566 - val_loss: 33.8842
Epoch 325/10000
5/5 [==============================] - 0s 4ms/step - loss: 33.5858 - val_loss: 38.7770
Epoch 326/10000
5/5 [==============================] - 0s 4ms/step - loss: 31.2956 - val_loss: 35.2207
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 32.3810 - val_loss: 32.3366
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 30.1050 - val_

...

5/5 [==============================] - 0s 5ms/step - loss: 10.8705 - val_loss: 20.1926
Epoch 2926/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.2024 - val_loss: 20.9963
Epoch 2927/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.8034 - val_loss: 24.6699
Epoch 2928/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.4459 - val_loss: 24.6670
Epoch 2929/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.2256 - val_loss: 21.8152
Epoch 2930/10000
5/5 [==============================] - 0s 4ms/step - loss: 10.2858 - val_loss: 16.0425
Epoch 2931/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.0282 - val_loss: 13.0502
Epoch 2932/10000
5/5 [==============================] - 0s 4ms/step - loss: 9.1815 - val_loss: 18.1730
Epoch 2933/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.1530 - val_loss: 12.8588
Epoch 2934/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.1590 - va

5/5 [==============================] - 0s 4ms/step - loss: 9.0318 - val_loss: 13.2540
Epoch 3005/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.5415 - val_loss: 19.1442
Epoch 3006/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.9438 - val_loss: 20.1282
Epoch 3007/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.3761 - val_loss: 15.1418
Epoch 3008/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.3425 - val_loss: 16.7008
Epoch 3009/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.8064 - val_loss: 14.2462
Epoch 3010/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.5277 - val_loss: 16.6354
Epoch 3011/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.5211 - val_loss: 14.8248
Epoch 3012/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.1928 - val_loss: 17.6534
Epoch 3013/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.1973 - val_l

5/5 [==============================] - 0s 5ms/step - loss: 9.3195 - val_loss: 23.6461
Epoch 3084/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.6898 - val_loss: 20.3192
Epoch 3085/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.1980 - val_loss: 15.3203
Epoch 3086/10000
5/5 [==============================] - 0s 4ms/step - loss: 10.2440 - val_loss: 15.3558
Epoch 3087/10000
5/5 [==============================] - 0s 4ms/step - loss: 9.6798 - val_loss: 18.3748
Epoch 3088/10000
5/5 [==============================] - 0s 4ms/step - loss: 8.6562 - val_loss: 18.1133
Epoch 3089/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.7135 - val_loss: 15.1629
Epoch 3090/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.2971 - val_loss: 12.0627
Epoch 3091/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.2596 - val_loss: 24.5358
Epoch 3092/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.2316 - val

5/5 [==============================] - 0s 7ms/step - loss: 6.9641 - val_loss: 25.7832
Epoch 3163/10000
5/5 [==============================] - 0s 5ms/step - loss: 14.8701 - val_loss: 23.1358
Epoch 3164/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.6104 - val_loss: 20.8589
Epoch 3165/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.7643 - val_loss: 28.1757
Epoch 3166/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.4500 - val_loss: 18.1444
Epoch 3167/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.8257 - val_loss: 18.7530
Epoch 3168/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.7304 - val_loss: 15.2086
Epoch 3169/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.4053 - val_loss: 13.6572
Epoch 3170/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.1180 - val_loss: 15.2867
Epoch 3171/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.8616 - va

5/5 [==============================] - 0s 5ms/step - loss: 8.2480 - val_loss: 21.1192
Epoch 3242/10000
5/5 [==============================] - 0s 4ms/step - loss: 6.1550 - val_loss: 22.2950
Epoch 3243/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1649 - val_loss: 12.9260
Epoch 3244/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.6353 - val_loss: 19.7923
Epoch 3245/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.3628 - val_loss: 14.4586
Epoch 3246/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.6664 - val_loss: 21.3643
Epoch 3247/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.9087 - val_loss: 21.6627
Epoch 3248/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.8756 - val_loss: 17.9720
Epoch 3249/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.0188 - val_loss: 28.8753
Epoch 3250/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.8726 - val_l

In [30]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [31]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,39.945782,39.904396,39.980965,39.950901,39.527946,37.082325,27.130018,26.559231,27.545528,32.076565,33.26149,39.635593,38.749794,33.015308,35.912861,36.910152,39.27776,39.782005,32.920116,31.940559,34.378754,31.204662,28.704891,25.782833,27.780899,33.493732,34.082222,40.081406,41.061504,44.215889
Target,45.71,45.564,43.754,48.554,40.896,35.255,34.018,22.422,29.555,28.919,25.589,40.237,37.012,38.793,38.281,39.711,33.695,28.206,31.615,30.275,29.568,26.94,35.91,33.954,37.983,40.046,32.647,39.065,36.18,28.749
Error,5.764217,5.659603,3.773037,8.6031,1.368053,1.827324,6.887983,4.13723,2.009472,3.157564,7.672489,0.601406,1.737793,5.777691,2.368137,2.800846,5.58276,11.576006,1.305117,1.66556,4.810753,4.264662,7.205109,8.171165,10.202103,6.552269,1.435223,1.016407,4.881504,15.466888


In [32]:
display(mae)
display(mape)

4.9427156

0.14720212

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[422.60074]] - Target[440.473]| =  Error: [[17.872253]]; MAPE:[[0.04057514]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[408.57968]] - Target[403.96]| =  Error: [[4.61969]]; MAPE:[[0.01143601]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[220.71567]] - Target[214.67]| =  Error: [[6.0456696]]; MAPE:[[0.02816262]]


[array([[17.872253]], dtype=float32),
 array([[4.61969]], dtype=float32),
 array([[6.0456696]], dtype=float32)]

9.512538

0.02672459